In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [2]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
#
n='Input_A3_017'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_A3_017
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.009448426063746636
test_rmse: 0.009560542910671769
test_r2: 0.4792472812657049
feature number: 3
train_rmse: 0.009205584329713887
test_rmse: 0.009355696027864895
test_r2: 0.5006511239690108
feature number: 4
train_rmse: 0.009110841653740622
test_rmse: 0.00927883664383767
test_r2: 0.5094779890548219
feature number: 5
train_rmse: 0.009021544601542492
test_rmse: 0.009185245312196517
test_r2: 0.5194517040383431
feature number: 6
train_rmse: 0.00900483950955693
test_rmse: 0.00920369698455193
test_r2: 0.5174761800487577
feature number: 7
train_rmse: 0.008961102956302812
test_rmse: 0.009208633165954964
test_r2: 0.5168433754335673
feature number: 8
train_rmse: 0.008959389068964114
test_rmse: 0.009223382322211114
test_r2: 0.515364223816105
feature number: 9
train_rmse: 0.008941102618453324
test_rmse: 0.009257937978939553
test_r2: 0.5117355726906383
feature number: 10
train_rmse: 0.008923969577419881
test_rmse: 0.009296639729868

train_rmse: 0.00887204281163348
test_rmse: 0.009416257836575095
test_r2: 0.49528157674258555
feature number: 16
train_rmse: 0.008851141584110364
test_rmse: 0.009447359932941048
test_r2: 0.4920349760493301
feature number: 17
train_rmse: 0.008841286519656634
test_rmse: 0.009488464187343516
test_r2: 0.4875367270661486
feature number: 18
train_rmse: 0.00882342932616345
test_rmse: 0.009493260208251253
test_r2: 0.48692320190304594
feature number: 19
train_rmse: 0.00881759228988293
test_rmse: 0.00952672617427177
test_r2: 0.48327018611120465
feature number: 20
train_rmse: 0.00874922595834042
test_rmse: 0.009556799163815344
test_r2: 0.48007747369201714
Test_rmse_min = 1.fea_num: 5  2.rmse: 0.009185245312196517
Test_r2_max = 1. fea_num: 5  2.r2: 0.5194517040383431
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.009448426063746636
test_rmse: 0.009560542910671769
test_r2: 0.4792472812657049
feature number: 3
train_rmse: 0.009205584329713887
test_rmse: 0.009355696027864895
test_r2: 0.5006511

train_rmse: 0.008941102618453324
test_rmse: 0.009257937978939553
test_r2: 0.5117355726906383
feature number: 10
train_rmse: 0.008923969577419881
test_rmse: 0.009296639729868657
test_r2: 0.5078325723394386
feature number: 11
train_rmse: 0.008897007319285406
test_rmse: 0.009309382589808332
test_r2: 0.506550658700678
feature number: 12
train_rmse: 0.00889371498550903
test_rmse: 0.00932915548214786
test_r2: 0.5045055141945847
feature number: 13
train_rmse: 0.0088891126714659
test_rmse: 0.009361623055272161
test_r2: 0.5010848568409433
feature number: 14
train_rmse: 0.008883757350174463
test_rmse: 0.009400884542760463
test_r2: 0.49691905839661565
feature number: 15
train_rmse: 0.00887204281163348
test_rmse: 0.009416257836575095
test_r2: 0.49528157674258555
feature number: 16
train_rmse: 0.008851141584110364
test_rmse: 0.009447359932941048
test_r2: 0.4920349760493301
feature number: 17
train_rmse: 0.008841286519656634
test_rmse: 0.009488464187343516
test_r2: 0.4875367270661486
feature number:

In [4]:
fea_number=5
#
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
#
y=data.Predict
skew_data=skew_pro(data)

num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A1_016', 'Input_A4_016', 'Input_A2_019',
       'Input_A1_018'], dtype=object)

## Linear Regression Baseline

In [9]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [10]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.009021544601542492
test_rmse: 0.009185245312196517
test_r2: 0.5194517040383431


## Lasso

In [11]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [12]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_x,y,parameters)

[{'alpha': 0.0009}, {'alpha': 0.001}, {'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.001}, {'alpha': 0.001}, {'alpha': 0.001}, {'alpha': 0.001}, {'alpha': 0.001}]


In [13]:
laso_rmse_ave(skew_x,y,0.0009)

test_rmse_ave: 0.009546973505247764
[0.010906597863411499, 0.009152790517823906, 0.010925003805371461, 0.009783225501728948, 0.01006344994625226, 0.008369120468723294, 0.010051092258568388, 0.00917646140553586, 0.008945635508242926, 0.008096357776819088]


test_r2_ave: 0.47177705418098326
[0.39273063673727837, 0.5225688447846071, 0.4171768959080201, 0.4711206391627524, 0.4647414840458818, 0.4736887520570783, 0.39636366332179174, 0.44640756622780475, 0.5557414378367435, 0.577230621727875]


In [14]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 9e-07}, {'alpha': 9e-05}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-05}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-05}]


In [15]:
laso_rmse_ave(skew_data[title],y,3e-07)

test_rmse_ave: 0.00915265946598786
[0.01062750198763127, 0.008673317787159884, 0.00994386176369197, 0.008939578064045368, 0.009945566589734396, 0.008430424097449754, 0.009745321367017846, 0.009006018749580881, 0.008403021442409763, 0.007811982811157472]


test_r2_ave: 0.5127135025130259
[0.42341258348195965, 0.5712795215682764, 0.5171594651920497, 0.5584025737690227, 0.4772080844813562, 0.4659500753571916, 0.4325322480300323, 0.46678132108470427, 0.6080015474223772, 0.6064076047432891]


## ElasticNet

In [16]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [18]:
alphas = [0.04]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_x,y,alphas,l1ratio)

alpha
[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]


l1_ratio
[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.03, 0.03, 0.02, 0.03]


In [19]:
ElasticNet_rmse_ave(skew_x,y,[0.04],0.025)

test_rmse_ave: 0.009593247243825615
[0.011043255441433449, 0.009252747275135597, 0.011074033473696565, 0.009869359895238074, 0.010033681305183917, 0.008346477943062217, 0.00992799086228947, 0.009222426494371726, 0.008974401193924147, 0.008188098553920995]


test_r2_ave: 0.4669254245596862
[0.37741736067365195, 0.5120839426540857, 0.40116768314896, 0.4617668240072028, 0.4679034914494219, 0.4765327534860977, 0.4110592655669314, 0.4408477640647468, 0.5528797185228036, 0.5675954420229605]


In [20]:
alphas = [0.04]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_data[title],y,alphas,l1ratio)

alpha
[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [21]:
ElasticNet_rmse_ave(skew_data[title],y,[0.04],0.01)

test_rmse_ave: 0.009230097048787068
[0.010712850925479455, 0.008792130351253567, 0.010074765589108081, 0.009202803075028248, 0.009927527433509496, 0.008270915189682867, 0.009711517230252325, 0.009112071378092598, 0.008554092848794324, 0.007942296466669701]


test_r2_ave: 0.5052574206935073
[0.41411430630033175, 0.5594533054152979, 0.504363289356982, 0.532014120812297, 0.4791028327153405, 0.48596800935175777, 0.43646223384147165, 0.4541492804609195, 0.5937799746249461, 0.5931668540557293]


## SVR

In [22]:
from sklearn.svm import SVR

In [23]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear', 'rbf','poly'], 'C':[1.2,1.3,1.1,1.4,1],'gamma': [1e-7,5e-6,5e-7],'epsilon':[0.2,0.1,0.05,0]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}




In [26]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear', 'rbf','poly'], 'C':[1.6,1.5,1.8,1.9,2],'gamma': [1e-3,1e-5,1e-4,1e-2],'epsilon':[0.1,0.05,0,0.2]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}




In [24]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    y = data['Predict']
    if fea_num != None:
        num=data.select_dtypes(exclude='object')
        numcorr=num.corr()
        title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
        X= data[title]
    else:
        X= data.drop(['Predict'], axis=1)
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [25]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.009154731992021709
test_rmse: 0.009323699506015254
test_r2: 0.5042389568202571


In [29]:
model=SVR(kernel='rbf', C=1.5, gamma= 1e-05, epsilon= 0)
svr_rmse_ave(skew_data,None,model)

train_rmse: 0.008550041959846303
test_rmse: 0.010438187159111215
test_r2: 0.3809993756689905


## XGB

In [5]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [46]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[04:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 70}


[04:02:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

In [32]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['gbtree'],
#     'n_estimators':[50,60,70],
    'n_estimators':range(50,221,10),
}

other_params = {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[12:03:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 80}


[12:03:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[12:04:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 70}


[12:04:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 80}


[12:04:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 220}


[12:05:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster':

### 調 max_depth / min_child_weight

In [48]:
# {'booster': 'dart', 'n_estimators': 60}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:04:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 4}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

In [34]:
# {'booster': 'dart', 'n_estimators': 160}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:22:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 6}


[12:22:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 3}


[12:22:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[12:22:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[12:22:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 6}


[12:22:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

### 調 gamma

In [49]:
# {'max_depth': 2, 'min_child_weight': 2}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:05:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



In [35]:
# {'max_depth': 7, 'min_child_weight': 3}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 2, 'min_child_weight': 4, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:23:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:23:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:23:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:23:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:23:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:24:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:24:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



### 調 subsample / colsample_bytree

In [51]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.3}


[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.9}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

In [6]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.4],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 2, 'min_child_weight': 4, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:34:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.7}


[12:34:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.9}


[12:34:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.7}


[12:34:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.9}


[12:34:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.8}


[12:34:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

### reg_alpha / reg_lambda

In [52]:
# {'colsample_bytree': 0.5, 'subsample': 0.6}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:10:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:27]

In [8]:
# {'colsample_bytree': 0.9, 'subsample': 0.7}

parameters_for_testing = {
#     'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
    'reg_alpha': [0.05],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 2, 'min_child_weight': 4, 'seed': 42,
 'gamma':0,'subsample':0.8,'colsample_bytree': 0.4}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:39:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda': 3}


[12:39:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda': 0.05}


[12:39:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda': 3}


[12:39:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda': 0.5}


[12:39:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda': 3}


[12:39:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.05, 'reg_lambda'

### learning rate

In [53]:
# {'reg_alpha': 0, 'reg_lambda': 0.7}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5,'reg_alpha': 0, 'reg_lambda': 0.7}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [9]:
# {'reg_alpha': 0.0, 'reg_lambda': 0.3}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'gbtree', 'n_estimators': 100, 'max_depth': 2, 'min_child_weight': 4, 'seed': 42,
 'gamma':0,'subsample':0.8,'colsample_bytree': 0.4,'reg_alpha': 0.05, 'reg_lambda': 2}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:40:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[12:40:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:40:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:40:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:40:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[12:40:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[12:40:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depr

In [131]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 60, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.85,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008496031540705907
[0.007648650629856374, 0.007861428699795805, 0.008923526006532567, 0.008422016710058338, 0.007514787893473913, 0.007671313900216453, 0.009908224384169528, 0.0094945394022681, 0.008777706642056635, 0.008738121138631347]


test_r2_ave: 0.5658204873179591
[0.6994093043442213, 0.6676009790074625, 0.49424336268772506, 0.5730727307503245, 0.6729951732468282, 0.6589820348516682, 0.3470225948052349, 0.46909149956225205, 0.5156770986673376, 0.5601100952565365]


In [10]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.4,
              reg_alpha =  0.05,
              reg_lambda = 2)
xgb_ave(skew_x, y,best_xgb_model)


test_rmse_ave: 0.009840044490529148
[0.011238638012964163, 0.009414674127418863, 0.010765569246852631, 0.009968603368226315, 0.010301925960229939, 0.008663731890335313, 0.010215688041430252, 0.009673836701100896, 0.009087854551093424, 0.009069923005639682]


test_r2_ave: 0.43822047569074796
[0.3551924161871589, 0.49485705126375545, 0.4340636840666695, 0.45088776013108733, 0.4390726029183243, 0.43598192625614607, 0.3764315971851053, 0.3847704808759226, 0.5415033736853427, 0.4694438643379676]


## RandomForest

In [11]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [78]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [79]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [81]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [20,40,50,60,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0688s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  64 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 108 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 134 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 190 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 220 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 254 tasks      | elapsed:    9.5s
[Parallel(n_jobs=6)]: Done 288 tasks      | elapse

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50}
best score
-6.733198261475197e-05


[Parallel(n_jobs=6)]: Done 1800 out of 1800 | elapsed:  1.4min finished


In [82]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    7.6s
[Parallel(

best params
{'max_leaf_nodes': 14, 'min_samples_leaf': 2}
best score
-6.728825706399575e-05


[Parallel(n_jobs=6)]: Done 585 out of 585 | elapsed:   24.0s finished


In [83]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 14,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.008359151931886303
[0.008046421179911343, 0.007522207654460264, 0.008970754200764595, 0.008192033878800358, 0.007261528804827015, 0.007389966886686526, 0.009434354784131698, 0.009095030814723662, 0.00923126977742949, 0.008447951337128053]


test_r2_ave: 0.5800138319443187
[0.6673317079609893, 0.6956681454215288, 0.4888757128331376, 0.5960708670050614, 0.6946648253743058, 0.6835371494626516, 0.40798747419915105, 0.5128303480153389, 0.46433192124209743, 0.5888401679289246]


In [12]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_x, y, test_size=0.2, random_state=42)

In [14]:
param_grid = {
              'n_estimators': [70,80,90],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    2.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.1s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    7.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    9.7s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   11.2s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   13.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   15.1s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   17.4s
[Parallel(

best params
{'max_depth': 3, 'min_samples_split': 3, 'n_estimators': 70}
best score
-8.674252187303733e-05


In [15]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 3,
            min_samples_split = 3,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    3.4s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.5s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    5.7s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    7.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    8.7s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   10.4s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   12.6s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   14.3s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   16.4s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   19.2s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   21.5s
[Parallel(

best params
{'max_leaf_nodes': 8, 'min_samples_leaf': 3}
best score
-8.64613953138218e-05


In [16]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 8,
            min_samples_leaf =3,
            random_state = 42) 
rand_ave(skew_x, y,rand_model)

test_rmse_ave: 0.009337846024906832
[0.010323443645221753, 0.009386344364815006, 0.010823630614421808, 0.008820585932611454, 0.010493764723731461, 0.00841560166896123, 0.009960122363808532, 0.008850865842604309, 0.008519864751579512, 0.0077842363413132545]


test_r2_ave: 0.4936121944141041
[0.45593354098007033, 0.4978925357832986, 0.42794275462119535, 0.5700802843286792, 0.4179873157807019, 0.4678263648099492, 0.40724093555822416, 0.48499531846761246, 0.5970243445324503, 0.6091985492788595]
